In [4]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
#import dataset CIFAR-10 from keras
from tensorflow.keras.datasets import cifar10

 Reason for Suitability: ResNet50’s depth and skip connections allow it to learn very fine-grained features, making it highly accurate for image classification. For small datasets like CIFAR-10, transfer learning allows us to leverage the pre-trained features from a large dataset (e.g., ImageNet) and fine-tune them for the task at hand, even with limited data.

In [13]:
#load the dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
#reduce the size of the dataset
x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

In [6]:
# load the ResNet50 model
width, height = 224, 224
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(width, height, 3))

In [7]:
# freeze the base model
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# create a new model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,441,802 (139.01 MB)

 Trainable params: 12,854,090 (49.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [9]:
# compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [10]:
import tensorflow as tf
import numpy as np

def image_resizer_generator(x, y, target_size=(224, 224)):
    while True:
        for i in range(0, len(x), 64):
            x_batch = x[i:i+64]
            y_batch = y[i:i+64]
            x_resized = np.array([tf.image.resize(x, target_size) for x in x_batch])
            yield x_resized, y_batch

# Create data generators
train_gen = image_resizer_generator(x_train, y_train, target_size=(224, 224))
val_gen = image_resizer_generator(x_test, y_test, target_size=(224, 224))




In [11]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(10000, 32, 32, 3)
(10000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [16]:
# train the model
model.fit(train_gen, steps_per_epoch=len(x_train) // 100, epochs=10, validation_data=val_gen, validation_steps=len(x_test) // 100)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 137s 14s/step - accuracy: 0.3642 - loss: 1.7380 - val_accuracy: 0.6359 - val_loss: 1.2428
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 137s 14s/step - accuracy: 0.4494 - loss: 1.6022 - val_accuracy: 0.6500 - val_loss: 1.1478
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 146s 15s/step - accuracy: 0.4744 - loss: 1.6143 - val_accuracy: 0.7016 - val_loss: 1.0175
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 141s 15s/step - accuracy: 0.5280 - loss: 1.5205 - val_accuracy: 0.6859 - val_loss: 1.0552
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 137s 14s/step - accuracy: 0.5419 - loss: 1.3394 - val_accuracy: 0.7031 - val_loss: 1.0032
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 141s 15s/step - accuracy: 0.5010 - loss: 1.4339 - val_accuracy: 0.6797 - val_loss: 1.0104
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 136s 14s/step - accuracy: 0.5769 - loss: 1.3155 - val_accuracy: 0.7484 - val_loss: 0.8218
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 133s 14s/step - accuracy: 0.5370 - loss: 1.4180 - val_accuracy: 0.

In [1]:
# evaluate the model
loss, acc = model.evaluate(val_gen, steps=len(x_test) // 64)
print('Accuracy: %.3f' % acc)


NameError: name 'model' is not defined